In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Read data in
file = np.load('output.npz')
xIn = file['x']
yIn = file['y']
print(xIn.shape)

(1, 1530, 2720, 3)


In [3]:
# setup the place holder input for the images
imageWidth = 1530
imageHeight = 2730

x = tf.placeholder(tf.float32, shape=[None, imageHeight, imageWidth, 3])

# setup placeholder input for labels
y = tf.placeholder(tf.float32, shape=[None, 2])

# placeholder for batch norm training phase.
trainPhase = tf.placeholder(tf.bool)

In [4]:
# define the batch norm function for use.
decayRate = 0.99
betaInit = tf.zeros_initializer(dtype=tf.float32)
gammaInit = tf.ones_initializer(dtype=tf.float32)

# batchNormLayer
# Adds a batch normalization layer to to the filter.
# x - input tensor
# filterShape - shape of filter
# num - the number to not have the same variable name for the gamma and beta variables.
# filtType - the type of filter (conv, mult)
def batchNormLayer(x, numChannels, num, filtType='conv'):
    # assumed to be convlution filter

    #define weight variables
    gamma = tf.get_variable('gamma' + str(num), [numChannels], initializer=gammaInit)
    beta = tf.get_variable('beta' + str(num), [numChannels], initializer=betaInit)

    axes = []
    if filtType == 'mult':
        axes = [0]
    else:
        axes = [0,1,2]
    #if len(filterShape) == 4:
    #    axes = [0,1,2]
    #else:
    #    axes = [0]
    
    batch_mean, batch_variance = tf.nn.moments(x, axes)
    
    ema = tf.train.ExponentialMovingAverage(decay=decayRate)
    #ema_apply_op = ema.apply([batch_mean, batch_variance])
    #mean = ema.average(batch_mean)
    #variance = ema.average(batch_variance)
    
    def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_variance])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_variance)

    mean, variance = tf.cond(trainPhase,
                        mean_var_with_update,
                        lambda: (ema.average(batch_mean), ema.average(batch_variance)))
    
    

    normed = tf.nn.batch_normalization(x, mean, variance, beta, gamma, 0.000001)
    return normed, gamma, beta, ema.average(batch_mean), ema.average(batch_variance)

In [5]:
numConvLayers = 0
# Define variable initilization
normInit = tf.truncated_normal_initializer(0,.05, dtype=tf.float32)
zeroInit = tf.constant_initializer(0.05, dtype=tf.float32)

# define convolutional layer
# x - the input tensor
# filterShape - the shape of the filter (height, width, num channels output)
# poolShape - the shape of the pooling (height width)
def convLayer(x, filterShape, poolShape):
    global numConvLayers
    inputChannels = x.shape[3]
    convFilt = tf.get_variable('filt' + str(numConvLayers), \
        [filterShape[0], filterShape[1], inputChannels, filterShape[2]], \
        initializer=normInit)
    bias = tf.get_variable('bias' + str(numConvLayers), \
        [filterShape[2]], initializer=normInit)
    
    logit = tf.nn.conv2d(x, convFilt, strides=[1,1,1,1], padding='SAME') + bias
    normed, gamma, beta, mean, variance = \
        batchNormLayer(logit, filterShape[2], numConvLayers)
    layer = tf.nn.relu(normed)
    pooled = tf.nn.max_pool(layer, \
                ksize=[1,poolShape[0], poolShape[1], 1], \
                strides=[1,poolShape[0], poolShape[1], 1], \
                padding='SAME')
    
    
    numConvLayers += 1
    return pooled, [convFilt, bias, gamma, beta, mean, variance]
    


In [7]:
variables = []

# define inference
layer1, tmp = convLayer(x, [3,3,128], [10,10])
variables += tmp
layer2, tmp = convLayer(layer1, [5,5,64], [3,3])
variables += tmp
print(layer2)

Tensor("MaxPool_2:0", shape=(?, 91, 51, 64), dtype=float32)
